In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

In [2]:
import numpy as np
import os
import cv2
face_cascade1 = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')
flag=0

In [3]:
subjects = ["", "TSP", "WMQ","YJD"]

def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow Haar classifier
    face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')

    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [4]:
def prepare_training_data(data_folder_path):
    
    dirs = os.listdir(data_folder_path)
    

    faces = []
   
    labels = []
    
    for dir_name in dirs:

        if not dir_name.startswith("s"):
            continue;
            
        
        label = int(dir_name.replace("s", ""))
        

        subject_dir_path = data_folder_path + "/" + dir_name
       
        subject_images_names = os.listdir(subject_dir_path)
        
     
        for image_name in subject_images_names:
           
            if image_name.startswith("."):
                continue;
            
            image_path = subject_dir_path + "/" + image_name

           
            image = cv2.imread(image_path)
            
            
            #detect face
            face, rect = detect_face(image)
            
            #------STEP-4--------
            #for the purpose of this tutorial
            #we will ignore faces that are not detected
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [5]:
faces, labels = prepare_training_data("training-data")
face_recognizer = cv2.face.createLBPHFaceRecognizer()
face_recognizer.train(faces, np.array(labels))

In [6]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

In [7]:
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [8]:
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)
    
    # print (face)
    
    if(face is None):
        return img
    #predict the image using our face recognizer
    else:
        label= face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
        label_text = subjects[label]
    
    #draw a rectangle around face detected
        draw_rectangle(img, rect)
    #draw name of predicted person
        draw_text(img, label_text, rect[0], rect[1]-5)
    
        return img

In [ ]:
cap = cv2.VideoCapture(0)
#获取第一帧
size1 = (160,120)
fx=4
fy=4

    

while(1):     
    if(base.switches[0].read() == 1 and base.switches[1].read() == 0):
        
        if(flag==0):
            ret, frame1 = cap.read()
            if(ret):
                frame1 = cv2.resize(frame1,size1,interpolation=cv2.INTER_AREA)
                prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
                hsv = np.zeros_like(frame1)
    #遍历每一行的第1列
                hsv[...,1] = 255
                flag=1
            else:
                raise RuntimeError("Failed to read from camera.")
        
        ret, frame2 = cap.read()
        if(ret):
            frame2 = cv2.resize(frame2,size1,interpolation=cv2.INTER_AREA)
            next1 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    #返回一个两通道的光流向量，实际上是每个点的像素位移值
            flow = cv2.calcOpticalFlowFarneback(prvs,next1, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    #print(flow.shape)
    #print(flow)

    #笛卡尔坐标转换为极坐标，获得极轴和极角
            mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
            hsv[...,0] = ang*180/np.pi/2
            hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
            rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
            rgb = cv2.resize(rgb, (0, 0), fx=fx, fy=fy, interpolation=cv2.INTER_CUBIC)
            outframe = hdmi_out.newframe()
            outframe[0:480,0:640,:] = rgb[0:480,0:640,:]
            hdmi_out.writeframe(outframe)

    #cv2.imshow('frame2',rgb)
    #k = cv2.waitKey(30) & 0xff
    #if k == 27:
    #    break
    #elif k == ord('s'):
    #    cv2.imwrite('opticalfb.png',frame2)
    #   cv2.imwrite('opticalhsv.png',rgb)
            prvs = next1
        else:
            raise RuntimeError("Failed to read from camera.")
            
    elif(base.switches[0].read() == 0 and base.switches[1].read() == 0):
    # cap = cv2.VideoCapture(0)
#size2 = (640,480)
        flag=0
        ret, frame = cap.read()
        if (ret):      
            frame = cv2.resize(frame,size1,interpolation=cv2.INTER_AREA)
    
            np_frame = frame
            gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade1.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2)

            frame = cv2.resize(frame, (0, 0), fx=fx, fy=fy, interpolation=cv2.INTER_CUBIC)
            outframe = hdmi_out.newframe()
            outframe[0:480,0:640,:] = frame[0:480,0:640,:]
            hdmi_out.writeframe(outframe)
        else:
            raise RuntimeError("Failed to read from camera.")

            
            
    elif(base.switches[0].read() == 1 and base.switches[1].read() == 1):
        flag=0
        ret, frame = cap.read()
        if (ret): 
            frame = cv2.resize(frame,size1,interpolation=cv2.INTER_AREA)
            predicted_img1 = predict(frame)
            predicted_img1 = cv2.resize(predicted_img1, (0, 0), fx=fx, fy=fy, interpolation=cv2.INTER_CUBIC)
        #plt.imshow(predicted_img1[:,:,[2,1,0]])
        #plt.show()
        #predicted_img1 = cv2.resize(predicted_img1,size1,interpolation=cv2.INTER_AREA)
            outframe = hdmi_out.newframe()
            outframe[0:480,0:640,:] = predicted_img1[0:480,0:640,:]
            hdmi_out.writeframe(outframe)
        #cv2.imshow(subjects[1], cv2.resize(predicted_img1, (640, 480)))
            
            
        else:
            raise RuntimeError("Failed to read from camera.")
            
    elif(base.switches[0].read() == 0 and base.switches[1].read() == 1):
        flag=0
        ret, frame = cap.read()
        if (ret): 
            outframe = hdmi_out.newframe()
        #cv2.Canny(frame, 100, 110, edges=outframe)
            laplacian_frame = cv2.Laplacian(frame, cv2.CV_8U, dst=outframe)
        #outframe[0:480,0:640,:] = frame[0:480,0:640,:]
            hdmi_out.writeframe(outframe)
        else:
            raise RuntimeError("Failed to read from camera.")
        
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break 
        
cap.release()